In [2]:
import os
import json
import torch
import logging 
import pandas as pd

from collections import defaultdict
from datasets import Dataset
import accelerate
import bitsandbytes

from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training, AdaLoraModel, AdaLoraConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, pipeline
from trl import SFTTrainer




In [3]:
def load_tokenized_dataset(self, name:str, file_path:str):
    data_dict = {}
    with open(file_path, 'r') as fp:
        id, questions, answers, text, input_id = json.load(fp)
        
        data_dict['id'] = id
        data_dict['questions'] = questions
        data_dict['answers'] = answers
        data_dict['text'] = text
        data_dict['input_ids'] = input_id
        
        self.dataset[name] = Dataset.from_dict(data_dict)
    
    return Dataset.from_dict(data_dict)

def load_model(base_model):
    base_model = AutoModelForCausalLM.from_pretrained(base_model)
    tokenizer = AutoTokenizer.from_pretrained(base_model)

    return base_model, tokenizer

def prepare_lora_model(base_model, lora_config):
    peft_model = prepare_model_for_kbit_training(base_model)
    lora_model = get_peft_model(peft_model, lora_config)
    
    return peft_model, lora_model

def prepare_adalora_model(base_model, adalora_config, name):
    return AdaLoraModel(base_model, adalora_config, name)

In [ ]:
gemma_model, gemma_tokenizer = load_model("google/gemma-7b")
llama2, llama2_tokenizer = load_model("meta-llama/Llama-2-7b-hf")
mistral_model, mistral_tokenizer = load_model("mistralai/Mistral-7B-v0.1")